# Askbot using PandasAI library

The motivation of the exercise is to parse the audio from the microphone to text, and and ask to OpenAI model what you said.

#### Imports 

I wrote every key i need to use the API's like pandasai or openai into a .env file in my root project directory, if u want to use the app you should make one or copy my method

In [5]:
import os
import pandas as pd
import pandasai
from pandasai import SmartDataframe
from pandasai.llm.openai import OpenAI
from dotenv import dotenv_values
#
import pyaudio
import wave
import audioop
import webrtcvad

# Keys
keys = dotenv_values("../../.env-token")

#### Global Parameters

In [6]:
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
THRESHOLD = 500
SILENCE_TIMEOUT = 4

#### Initialization


In [9]:
# Environment key
os.environ["PANDASAI_API_KEY"] = keys["PANDASAI_API_KEY"]
llm = OpenAI(api_token=keys["OPENAI_API_KEY"])
# Pyaudio
audio = pyaudio.PyAudio()

#### Record

In [7]:
# This will record audio till we stay 4 seconds without talk

stream = audio.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    frames_per_buffer=CHUNK
)
print("You can talk now.\nRecording...")
#
frames = []
silence_counter = 0
vad = webrtcvad.Vad()
vad.set_mode(3) # <-- Aggressive detection to record voice

while True:
    data = stream.read(CHUNK)
    frames.append(data)
    is_speech = vad.is_speech(data, RATE)
    if not is_speech:
        silence_counter += 1
    else:
        silence_counter = 0
    if silence_counter >= int( SILENCE_TIMEOUT * ( RATE / CHUNK )):
        break

print("\nFinished recording.")

# Stop stream
stream.stop_stream()
stream.close()

# Stop PyAudio
audio.terminate()

# Save the recorded audio as .wav file
with wave.open("recorded_audio.wav", "wb") as audio_recorded:
    audio_recorded.setnchannels(CHANNELS)
    audio_recorded.setsampwidth(audio.get_sample_size(FORMAT))
    audio_recorded.setframerate(RATE)
    audio_recorded.writeframes(b"".join(frames))

After we record and saved the audio we still have to process it with speech2text and ask the question to OpenAI.

When we complete every step, the we would just have convert to speech the generated text and play it (text2speech)